In [1]:
import os
import glob
import rdflib
import concurrent.futures

# Define a function to read and parse a single JSON-LD file into an RDFLib graph
def read_and_parse_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
        graph = rdflib.Graph()
        graph.parse(data=file_content, format='json-ld')
    return graph

# Define a function to load JSON-LD files into an RDFLib graph using concurrent processing
def load_jsonld_files_to_graph(folder_path):
    # Get all JSON-LD files in the specified folder
    jsonld_files = glob.glob(os.path.join(folder_path, "*.json"))

    # Create an empty RDFLib graph
    graph = rdflib.Graph()

    # Use concurrent processing to read and parse files in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the read_and_parse_file function to each file path
        results = executor.map(read_and_parse_file, jsonld_files)

    # Combine all graphs into one
    for g in results:
        graph += g

    return graph

In [2]:
# Define the folder containing the JSON-LD files
folder_path = 'BatteryTypeJson'

# Load JSON-LD files into the graph
graph = load_jsonld_files_to_graph(folder_path)

# Print the number of triples in the graph
print(f"Graph has {len(graph)} triples.")

# Optionally, serialize the graph to a file (e.g., in Turtle format)
#output_file = 'output_graph.ttl'
#graph.serialize(destination=output_file, format='turtle')
#print(f"Serialized graph to {output_file}")


Graph has 37281 triples.


In [3]:
# Define the SPARQL query
query = """
PREFIX schema: <https://schema.org/>

SELECT ?name (COUNT(?subjectOf) AS ?subjectOfCount)
WHERE {
  ?thing schema:subjectOf ?subjectOf .
  ?thing schema:name ?name .
}
GROUP BY ?name
ORDER BY DESC(?subjectOfCount)
LIMIT 10
"""

# Execute the query
qres = graph.query(query)

# Print the results
for row in qres:
    print(f"{row.name} is the subject of {row.subjectOfCount} publications")

NCR18650BF is the subject of 62 publications
INR18650 MJ1 is the subject of 44 publications
INR21700 M50T is the subject of 18 publications
SLPB11543140H5 is the subject of 16 publications
AMP20M1HD-A is the subject of 12 publications
INR21700 M50 is the subject of 12 publications
INR21700-50E is the subject of 11 publications
SLPB75106100 is the subject of 11 publications
WB-LYP40AHA is the subject of 6 publications
SLPB120216216HR2 is the subject of 6 publications


In [9]:
import requests
# Define the SPARQL query
query = """
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
    ?thing schema:name "20AH" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "A123".
    ?thing <{hasProperty}> ?property .
    ?property a <{RatedCapacity}> ;
              <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
}}
"""
sparql_query = """
    SELECT ?value
    WHERE {{
    ?thing schema:name "INR21700 M50T" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "LG Chem".
    ?thing <{hasProperty}> ?property .
    ?property a <{RatedCapacity}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
    }}
"""
    
# Execute the query
#qres = graph.query(query)

# Print the results
#for row in qres:
#    print(f"{row}")

context_url = "https://w3id.org/emmo/domain/battery/context"
context_data = requests.get(context_url).json()

In [11]:
qres = graph.query(add_Iri_context(sparql_query,context_data))
for row in qres:
    print(f"{row}")

(rdflib.term.Literal('4.8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')),)


In [7]:
def add_Iri_context(query,context_data):

    # Extract IRIs from the context file
    hasPositiveElectrode = rdflib.URIRef(context_data["@context"]["hasPositiveElectrode"]["@id"])
    PositiveElectrode = rdflib.URIRef(context_data["@context"]["PositiveElectrode"])
    hasActiveMaterial = rdflib.URIRef(context_data["@context"]["hasActiveMaterial"]["@id"])
    NMC = rdflib.URIRef(context_data["@context"]["LithiumNickelManganeseCobaltOxide"])
    LFP = rdflib.URIRef(context_data["@context"]["LithiumIronPhosphate"])
    hasProperty = rdflib.URIRef(context_data["@context"]["hasProperty"]["@id"])
    hasNumericalPart = rdflib.URIRef(context_data["@context"]["hasNumericalPart"]["@id"])
    hasNumericalValue = rdflib.URIRef(context_data["@context"]["hasNumericalValue"])
    RatedCapacity = rdflib.URIRef(context_data["@context"]["RatedCapacity"])
    CycleLife = rdflib.URIRef(context_data["@context"]["CycleLife"])
    NominalVoltage = rdflib.URIRef(context_data["@context"]["NominalVoltage"])
    UpperVoltageLimit = rdflib.URIRef(context_data["@context"]["UpperVoltageLimit"])
    LowerVoltageLimit = rdflib.URIRef(context_data["@context"]["LowerVoltageLimit"])
    DischargingCurrent = rdflib.URIRef(context_data["@context"]["DischargingCurrent"])
    MaximumContinuousDischargingCurrent = rdflib.URIRef(context_data["@context"]["MaximumContinuousDischargingCurrent"])
    MaximumContinuousChargingCurrent = rdflib.URIRef(context_data["@context"]["MaximumContinuousChargingCurrent"])
    Mass = rdflib.URIRef(context_data["@context"]["Mass"])
    ChargingCurrent = rdflib.URIRef(context_data["@context"]["ChargingCurrent"])
    Height = rdflib.URIRef(context_data["@context"]["Height"])
    Diameter = rdflib.URIRef(context_data["@context"]["Diameter"])

    return query.format(
    hasPositiveElectrode=str(hasPositiveElectrode),
    PositiveElectrode=str(PositiveElectrode),
    hasActiveMaterial=str(hasActiveMaterial),
    NMC=str(NMC),
    LFP=str(LFP),
    hasProperty=str(hasProperty),
    hasNumericalPart=str(hasNumericalPart),
    hasNumericalValue=str(hasNumericalValue),
    RatedCapacity=str(RatedCapacity),
    CycleLife=str(CycleLife),
    NominalVoltage=str(NominalVoltage),
    UpperVoltageLimit=str(UpperVoltageLimit),
    LowerVoltageLimit=str(LowerVoltageLimit),
    DischargingCurrent=str(DischargingCurrent),
    MaximumContinuousDischargingCurrent=str(MaximumContinuousDischargingCurrent),
    MaximumContinuousChargingCurrent=str(MaximumContinuousChargingCurrent),
    Mass=str(Mass),
    ChargingCurrent=str(ChargingCurrent),
    Height=str(Height),
    Diameter=str(Diameter))

In [5]:
import requests

# Load the context file from the URL
context_url = "https://w3id.org/emmo/domain/battery/context"
context_data = requests.get(context_url).json()

# Extract IRIs from the context file
hasPositiveElectrode = rdflib.URIRef(context_data["@context"]["hasPositiveElectrode"]["@id"])
PositiveElectrode = rdflib.URIRef(context_data["@context"]["PositiveElectrode"])
hasActiveMaterial = rdflib.URIRef(context_data["@context"]["hasActiveMaterial"]["@id"])
NMC = rdflib.URIRef(context_data["@context"]["LithiumNickelManganeseCobaltOxide"])
LFP = rdflib.URIRef(context_data["@context"]["LithiumIronPhosphate"])
hasProperty = rdflib.URIRef(context_data["@context"]["hasProperty"]["@id"])
hasNumericalPart = rdflib.URIRef(context_data["@context"]["hasNumericalPart"]["@id"])
hasNumericalValue = rdflib.URIRef(context_data["@context"]["hasNumericalValue"])
RatedCapacity = rdflib.URIRef(context_data["@context"]["RatedCapacity"])
CycleLife = rdflib.URIRef(context_data["@context"]["CycleLife"])
NominalVoltage = rdflib.URIRef(context_data["@context"]["NominalVoltage"])
UpperVoltageLimit = rdflib.URIRef(context_data["@context"]["UpperVoltageLimit"])
LowerVoltageLimit = rdflib.URIRef(context_data["@context"]["LowerVoltageLimit"])
DischargingCurrent = rdflib.URIRef(context_data["@context"]["DischargingCurrent"])
MaximumContinuousDischargingCurrent = rdflib.URIRef(context_data["@context"]["MaximumContinuousDischargingCurrent"])
Mass = rdflib.URIRef(context_data["@context"]["Mass"])
ChargingCurrent = rdflib.URIRef(context_data["@context"]["ChargingCurrent"])
Height = rdflib.URIRef(context_data["@context"]["Height"])
Diameter = rdflib.URIRef(context_data["@context"]["Diameter"])

# Define the SPARQL query using the extracted IRIs
query = f"""
SELECT (COUNT(?thing) AS ?countInstance)
WHERE {{
  ?thing <{hasPositiveElectrode}> ?positiveElectrode .
  ?positiveElectrode a <{PositiveElectrode}> ;
                     <{hasActiveMaterial}> ?activeMaterial .
  ?activeMaterial a <{NMC}> .
}}
"""

# Execute the query
qres = graph.query(query)

# Print the results
for row in qres:
    print(f"Count of cells with NMC active materials: {row.countInstance}")

Count of cells with NMC active materials: 76


In [6]:
name_search="NCR18650BF"
query = f"""
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
  ?thing schema:name "{name_search}" .
  ?thing <{hasProperty}> ?property .
  ?property a <{RatedCapacity}> ;
            <{hasNumericalPart}> ?numericalPart .
  ?numericalPart <{hasNumericalValue}> ?value .
}}
"""

# Execute the query
qres = graph.query(query)

# Print the results
for row in qres:
    print(row.value)

3.35


In [7]:
name_search="NCR18650BF"
query3 = f"""
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
  ?thing schema:name "{name_search}" .
  ?thing <{hasProperty}> ?property .
  ?property a <{NominalVoltage}> ;
            <{hasNumericalPart}> ?numericalPart .
  ?numericalPart <{hasNumericalValue}> ?value .
}}
"""

# Execute the query
qres = graph.query(query3)

# Print the results
for row in qres:
    print(row.value)

3.6


In [13]:
# name_search = "NCR18650BF"
#name_search = "INR21700 M50T"
#query = f"""
# PREFIX schema: <https://schema.org/>
# SELECT ?value
# WHERE {{
#     ?thing schema:name "CA100" .
#     ?thing schema:manufacturer ?manufacturer.
#     ?manufacturer schema:name "CALB".
#     ?thing <{hasProperty}> ?property .
#     ?property a <{ChargingCurrent}> ;
#               <{hasNumericalPart}> ?numericalPart .
#     ?numericalPart <{hasNumericalValue}> ?value .
# }}

# """

query = f"""
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
    ?thing schema:name "CA100" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "CALB".
    ?thing <{hasProperty}> ?property .
    ?property a <{ChargingCurrent}> ;
              <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
}}
"""
# Execute the query
qres = graph.query(query)

# Print the results
for row in qres:
    print(row.value)
    #print(name_search + ' is thing ' + row.thing)

30.0
